## 데이터 전처리 (Data Preprocessing)

In [1]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter

In [2]:
# data 로드하기(train, test), length 출력
# 'ratings_train.txt', ratings_test.txt'
train_df = pd.read_table('./data/ratings_train.txt')
test_df = pd.read_table('./data/ratings_test.txt')
print(train_df.head())
print("=========================================================================================================")
print(test_df.head())
print("=========================================================================================================")
# 중복값 확인
print(train_df['document'].nunique())
print("=========================================================================================================")
# 라벨 별 count
print(train_df.groupby('label').size().reset_index(name='count'))
print("=========================================================================================================")
train_df.isnull().values.any()

         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0
146182
   label  count
0      0  75173
1      1  74827


True

In [3]:
# 데이터 살펴보기 (컬럼명, 중복항, 라벨 별 count, Null 값 유무)
train_df.info()
test_df.info()
print(train_df['document'].duplicated().sum())
print(test_df['document'].duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB
3817
842


In [4]:
# Train data
train_data = pd.read_table('./data/ratings_train.txt')
# Document 컬럼의 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
print(len(train_data))
# Document 컬럼의 Null 값 제거
train_data = train_data.dropna(how='any')
print(train_data.isnull().values.any())
print(len(train_data))

146183
False
146182


In [5]:
# Document 컬럼의 한글과 공백 텍스트를 제외하고 모두 제거
# 한글 추출 정규식: "[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"
import re
train_data['document'] = train_data['document'].apply(lambda x: re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", x))
train_data

,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
1,3819312,흠포스터보고초딩영화줄오버연기조차가볍지않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소이야기구먼솔직히재미는없다평점조정,0
4,6483659,사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이...,1
...,...,...,...
149995,6222902,인간이문제지소는뭔죄인가,0
149996,8549745,평점이너무낮아서,1
149997,9311800,이게뭐요한국인은거들먹거리고필리핀혼혈은착하다,0
149998,2376369,청춘영화의최고봉방황과우울했던날들의자화상,1


In [6]:
# 강사님ver
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", regex=True)
train_data[:5]

,id,document,label
0,9976970,아더빙진짜짜증나네요목소리,0
1,3819312,흠포스터보고초딩영화줄오버연기조차가볍지않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소이야기구먼솔직히재미는없다평점조정,0
4,6483659,사이몬페그의익살스런연기가돋보였던영화스파이더맨에서늙어보이기만했던커스틴던스트가너무나도이...,1


In [8]:
# 강사님ver
train_data['document'] = train_data['document'].str.replace("^ +", "").replace('', pd.NA, regex=False)
train_data['document'].replace("", np.nan, inplace=True)
print(train_data.isnull().sum())

# Null 값 제거
train_data = train_data.dropna(how='any')
print("최종 train 데이터 샘플 수 : ", len(train_data))

id          0
document    0
label       0
dtype: int64
최종 train 데이터 샘플 수 :  145393


C:\Users\BIG3-06\AppData\Local\Temp\ipykernel_17656\1191677189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['document'] = train_data['document'].str.replace("^ +", "").replace('', pd.NA, regex=False)
C:\Users\BIG3-06\AppData\Local\Temp\ipykernel_17656\1191677189.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['document'].replace("", np.nan, inplace=True)


In [9]:
# Test 데이터 적용
test_data = pd.read_table('./data/ratings_test.txt')
# Document 컬럼의 중복 제거
test_data.drop_duplicates(subset=['document'], inplace=True)
print(len(train_data))
# Document 컬럼의 Null 값 제거
test_data = test_data.dropna(how='any')
print(test_data.isnull().values.any())
print(len(test_data))
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", regex=True)
test_data['document'] = test_data['document'].str.replace("^ +", "").replace('', pd.NA, regex=False)
test_data['document'].replace("", np.nan, inplace=True)
print(train_data.isnull().sum())

# Null 값 제거
test_data = test_data.dropna(how='any')
print("최종 train 데이터 샘플 수 : ", len(test_data))

145393
False
49157
id          0
document    0
label       0
dtype: int64
최종 train 데이터 샘플 수 :  48852


In [10]:
# 불용어
stopwords = ['은', '는', '이', '가', '을', '를', '의', '한', '에', '하', '고', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '도', '는', '다']

In [11]:
# train 데이터 형태소 분석 및 전처리: 4000개만
X_train = []

okt = Okt()

for sentence in train_data['document'][:4000]:
    # 형태소 단위
    tokenized_word = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_word if word not in stopwords]
    X_train.append(stopwords_removed_sentence)
   

In [ ]:
# # Mecab 사용을 원할 시 colab 권장
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab_light_220429.sh
# from konlpy.tag import Mecab
# mecab = Mecab()
# mecab.morphs("안녕하세요~")

In [12]:
# 전처리 된 train 데이터 출력
print(X_train[:5])

[['아더', '빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천'], ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정'], ['사이', '몬페', '그', '익살스럽다', '연기', '돋보이다', '영화', '스파이더맨', '에서', '늙다', '보이다', '하다', '크다', '스틴던스트', '너무나도', '이쁘다', '보이다']]


In [13]:
# test 데이터 형태소 분석 및 전처리: 2000개만
X_test = []

okt = Okt()

for sentence in test_data['document'][:2000]:
    # 형태소 단위
    tokenized_word = okt.morphs(sentence, stem=True)
    stopwords_removed_sentence = [word for word in tokenized_word if word not in stopwords]
    X_test.append(stopwords_removed_sentence)

In [14]:
# 전처리 된 test 데이터 출력
print(X_test[:5])

[['굳다', 'ㅋ'], ['뭐', '야', '평점', '나쁘다', '않다', '만점', '짜다', '리', '더', '더욱', '아니다'], ['지루하다', '않다', '완전', '막장', '임돈', '주다', '보기', '에는'], ['만', '아니다', '별', '다섯', '개', '주다', '왜', '로나', '서다', '제', '심기', '불편하다'], ['음악', '주가', '되다', '최고', '음악', '영화']]


In [15]:
# 데이터를 나눈 대로 라벨도 맟춰주기
y_train = np.array(train_data['label'][:4000])
y_test = np.array(test_data['label'][:2000])

# train 데이터를 train, valid 데이터로 나누기 (8:2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=777, stratify=y_train)

In [16]:
# Train, valid, test 별 클래스(부정, 긍정) 비율(%) 출력하기 (Train, valid, test)
print("Train 데이터 클래스 비율")
print(f'부정 = {round(np.sum(y_train==0)/len(y_train)*100, 3)}%')
print(f'긍정 = {round(np.sum(y_train==1)/len(y_train)*100, 3)}%')
print("Valid 데이터 클래스 비율")
print(f'부정 = {round(np.sum(y_valid==0)/len(y_valid)*100, 3)}%')
print(f'긍정 = {round(np.sum(y_valid==1)/len(y_valid)*100, 3)}%')
print("Test 데이터 클래스 비율")
print(f'부정 = {round(np.sum(y_test==0)/len(y_test)*100, 3)}%')
print(f'긍정 = {round(np.sum(y_test==1)/len(y_test)*100, 3)}%')

Train 데이터 클래스 비율
부정 = 50.062%
긍정 = 49.938%
Valid 데이터 클래스 비율
부정 = 50.125%
긍정 = 49.875%
Test 데이터 클래스 비율
부정 = 48.75%
긍정 = 51.25%


In [17]:
# 전체 단어수(Unique) 세기 : X_train 기준
# 리뷰의 최대 길이
word_len = [len(l) for l in X_train]
print(f"가장 긴 리뷰 길이: {max(word_len)}")


가장 긴 리뷰 길이: 54


In [18]:
# 빈도수 기준으로 상위 10개 출력

# 강사님ver
word_list = []
for sent in X_train:
    for word in sent:
        word_list.append(word)

# word_counts = len(list(set(word_list)))
word_counts = Counter(word_list)
print("전체 voca 수: ", len(word_counts))

# 빈도
vocab = sorted(word_counts, key=word_counts.get, reverse=True)
print(vocab[:10])

전체 voca 수:  7161
['영화', '보다', '하다', '없다', '이다', '좋다', '정말', '재밌다', '같다', '되다']


In [19]:
# 단어 수, 등장 빈도 N번 미만 단어 수, 비율 등 계산 및 출력
threshold = 3
total_cnt = len(word_counts) # 전체 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

for key, value in word_counts.items():
    total_freq += value
    if (value < threshold):
        rare_cnt += 1
        rare_freq += value

print("단어 voca의 크기 :", total_cnt)
print("등장 빈도가 N 이하인 단어의 수:", rare_cnt)
print("전체 단어 중 등장 빈도가 N 이하인 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 N 이하인 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 voca의 크기 : 7161
등장 빈도가 N 이하인 단어의 수: 5185
전체 단어 중 등장 빈도가 N 이하인 단어의 비율: 72.40608853512079
전체 등장 빈도에서 N 이하인 단어 등장 빈도 비율: 18.255702176002796


In [20]:
# vocab에서 빈도수 threshold 이하인 단어는 제거, 단어수 출력
# 등장 빈도가 threshold 이상인 단어만 남기고 제거
vocab_size = total_cnt - rare_cnt
vocab = vocab[:vocab_size]
print("최종 단어 수 : ", len(vocab))

최종 단어 수 :  1976


In [21]:
# Voca dictionary 구성하기
# Speical token
word_to_index = {}
word_to_index['<PAD>'] = 0
word_to_index['<UNK>'] = 1

# 단어 dictionary 만들기 {pad:0, unk:1, word:2, word:3}
for index, word in enumerate(vocab):
    word_to_index[word] = index + 2
vocab_size = len(word_to_index)
print('총 단어수 :', len(word_to_index))
print(word_to_index)

총 단어수 : 1978
{'<PAD>': 0, '<UNK>': 1, '영화': 2, '보다': 3, '하다': 4, '없다': 5, '이다': 6, '좋다': 7, '정말': 8, '재밌다': 9, '같다': 10, '되다': 11, '만': 12, '너무': 13, '있다': 14, '진짜': 15, '적': 16, '으로': 17, '아니다': 18, '않다': 19, '로': 20, '내': 21, '에서': 22, '평점': 23, '최고': 24, '나': 25, '사람': 26, '점': 27, '그': 28, '연기': 29, '이영화': 30, '나오다': 31, '스토리': 32, '드라마': 33, '생각': 34, '시간': 35, '하고': 36, '아깝다': 37, '재미없다': 38, '감동': 39, '만들다': 40, '것': 41, '배우': 42, '때': 43, '요': 44, 'ㅋㅋ': 45, '들다': 46, '그냥': 47, '감독': 48, '왜': 49, '이런': 50, '보고': 51, '재미있다': 52, '재미': 53, '아': 54, '가다': 55, '뭐': 56, '주다': 57, '말': 58, '못': 59, '정도': 60, '그렇다': 61, '거': 62, '모르다': 63, '작품': 64, '지루하다': 65, '면': 66, '주인공': 67, '쓰레기': 68, '까지': 69, '오다': 70, '알다': 71, '사랑': 72, '본': 73, '중': 74, '이나': 75, 'ㅠㅠ': 76, '화': 77, '더': 78, '좀': 79, '마지막': 80, 'ㅋ': 81, '자다': 82, '싶다': 83, '이렇게': 84, '년': 85, '돈': 86, '저': 87, '기': 88, '라': 89, '대': 90, '남자': 91, 'ㅡㅡ': 92, '볼': 93, '역시': 94, '액션': 95, '연출': 96, '넘다': 97, '남다': 98, '영': 99, '

In [22]:
# 정수 인코딩 method(if문)
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sentence in tokenized_X_data:
    encoded_sentence = [] # 현재 문장의 정수 인코딩된 결과를 저장할 리스트
    for word in sentence:
      # 단어가 word_to_index에 있으면 해당 인덱스를 가져오고, 없으면 <UNK>의 인덱스를 사용
      if word in word_to_index:
          encoded_sentence.append(word_to_index[word])
      else:
          encoded_sentence.append(word_to_index['<UNK>'])
    encoded_X_data.append(encoded_sentence)
  return encoded_X_data

In [ ]:
# 정수 인코딩 method(try문)
def texts_to_sequences(tokenized_X_data, word_to_index):
    encoded_X_data = []    
    for sent in tokenized_X_data:
        index_sequences = []
        for word in sent:
            try:
                index_sequences.append(word_to_index[word])
            except:
                index_sequences.append(word_to_index['<UNK>'])
        encoded_X_data.append(index_sequences)
    return encoded_X_data

In [23]:
# Train, valid, test 데이터를 정수 인코딩 하여 변환
encoded_X_train = texts_to_sequences(X_train, word_to_index)
encoded_X_valid = texts_to_sequences(X_valid, word_to_index)
encoded_X_test = texts_to_sequences(X_test, word_to_index)

In [24]:
# 인코딩 된 문장 출력해 보기
print(encoded_X_train[1])

[30, 15, 1486, 310, 2]


In [26]:
# 디코딩 Dictionary 만들기
# word_to_index의 reverse된 dictionary 구성: {idx: words}
index_to_word = {}

# 단어 dictionary 만들기 {pada:0, unk:1, word:2, word:3}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [27]:
# 인코딩 된 문장 디코딩 하기, 출력
decoded_sample = [index_to_word[word] for word in encoded_X_train[1]]

#sample
print(X_train[1])

# decoding
print(decoded_sample)

['이영화', '진짜', '제이슨', '욕', '영화']
['이영화', '진짜', '제이슨', '욕', '영화']


In [28]:
# padding : 정해진 길이(max_length)에 따라 문서의 길이를 맞춰주는(split or padding) 과정
print('문서의 최대 길이 :', max([len(review) for review in encoded_X_train]))
print('문서의 최th 길이 :', min([len(review) for review in encoded_X_train]))
print('문서의 평균 길이 :', sum(map(len, encoded_X_train)) / len(encoded_X_train))

문서의 최대 길이 : 54
문서의 최th 길이 : 1
문서의 평균 길이 : 10.7278125


In [29]:
# 전체 샘플 중 길이가 max_length 이하인 샘플의 비율(X_train 기준)
max_len = 30 # 임의의 max_length 지정
count = 0

# for i in X_train:
#     if len(i) <= max_len:
#         count += 1

for sentence in X_train:
    if (len(sentence) <= max_len):
        count += 1

print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(X_train))*100))

전체 샘플 중 길이가 30 이하인 샘플의 비율: 94.09375


In [30]:
# train, valid, test에 대해 padding 진행: max_length에 맞춰 단어가 없는 구간에 0을 채워 넣는다
def pad_sequences(sentences, max_len):

  return features

print('train 데이터의 크기 :', )
print('valid 데이터의 크기 :', )
print('test 데이터의 크기 :', )

train 데이터의 크기 :
valid 데이터의 크기 :
test 데이터의 크기 :


In [31]:
def pad_sequences(sentences, max_len):
    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[index, :len(sentence)]= np.array(sentence)[:max_len]
    return features

# train, valid, test 데이터에 대해 패딩 진행
padded_X_train = pad_sequences(encoded_X_train, max_len=max_len)
padded_X_valid = pad_sequences(encoded_X_valid, max_len=max_len)
padded_X_test = pad_sequences(encoded_X_test, max_len=max_len)

print('train 데이터의 크기 :', padded_X_train.shape)
print('valid 데이터의 크기 :', padded_X_valid.shape)
print('test 데이터의 크기 :', padded_X_test.shape)

train 데이터의 크기 : (3200, 30)
valid 데이터의 크기 : (800, 30)
test 데이터의 크기 : (2000, 30)


In [32]:
# 패딩 처리 된 데이터 출력해 보기
print(len(padded_X_train[0]))
print(padded_X_train[0])

30
[  1 660 492 493 754 533  91  67   3  47 451  75 110 309   2 853   1   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [33]:
def pad_sequences_stu(sentences, max_len):
    features = []
    for sentence in sentences:
        # 문장의 길이가 max_len보다 짧으면
        if len(sentence) < max_len:
            # 부족한 부분을 0으로 채워줌
            padded_sentence = sentence + [0] * (max_len - len(sentence))
        # 문장의 길이가 max_len보다 길면
        else:
            # max_len까지만 자름
            padded_sentence = sentence
        features.append(padded_sentence)
    return features

# train, valid, test 데이터에 대해 패딩 진행
X_train_padded_stu = pad_sequences_stu(encoded_X_train, max_len)
X_valid_padded_stu = pad_sequences_stu(encoded_X_valid, max_len)
X_test_padded_stu = pad_sequences_stu(encoded_X_test, max_len)

print('train 데이터의 크기 :', len(X_train_padded_stu))
print('valid 데이터의 크기 :', len(X_valid_padded_stu))
print('test 데이터의 크기 :', len(X_test_padded_stu))

train 데이터의 크기 : 3200
valid 데이터의 크기 : 800
test 데이터의 크기 : 2000


In [34]:
# 패딩 처리 된 데이터 출력해 보기
print(X_train_padded_stu[0])

[1, 660, 492, 493, 754, 533, 91, 67, 3, 47, 451, 75, 110, 309, 2, 853, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
